In [ ]:
# Claims to be faster than mobilenet 
# https://gist.github.com/imbibekk/fde87f0a57f351d28e182828d118c98b # No direct keras implementation

In [32]:
import tensorflow as tf

from tensorflow.keras.layers import Input , Dense , Flatten , GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from pathlib import Path
from tensorflow.keras import layers
from keras_preprocessing.image import ImageDataGenerator
import pandas as pd
from sklearn.model_selection import train_test_split

In [36]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, SeparableConv2D,BatchNormalization,concatenate, Input,AveragePooling2D
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.models import Model


def _DenseLayer(x,num_input_features, growth_rate, bottleneck_width, drop_rate):

    growth_rate = int(growth_rate / 2)
    inter_channel = int(growth_rate  * bottleneck_width / 4) * 4 

    if inter_channel > num_input_features / 2:
        inter_channel = int(num_input_features / 8) * 4
        print('adjust inter_channel to ',inter_channel)


    left = Conv2D(inter_channel, (1, 1), padding='same')(x)
    left = BatchNormalization()(x)
    left = Activation('relu')(x)

    left = Conv2D(growth_rate, (3, 3), padding='same')(x)
    left = BatchNormalization()(x)
    left = Activation('relu')(x)

    right = Conv2D(inter_channel, (1, 1), padding='same')(x)
    right = BatchNormalization()(x)
    right = Activation('relu')(x)

    right = Conv2D(growth_rate, (3, 3), padding='same')(x)
    right = BatchNormalization()(x)
    right = Activation('relu')(x)

    right = Conv2D(growth_rate, (3, 3), padding='same')(x)
    right = BatchNormalization()(x)
    right = Activation('relu')(x)
    
    out = concatenate([left,x,right],axis=3)

    return out


def basic_conv(x, output_channels, kernel_size, stride):
    x = Conv2D(output_channels, (kernel_size, kernel_size),strides=(stride,stride), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x


def _DenseBlock(x, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        
        for i in range(num_layers):
                x = _DenseLayer(x, num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)

        return x



def _StemBlock(x, num_init_features):

    num_stem_features = int(num_init_features/2)
    
    out = basic_conv(x, num_init_features, kernel_size=3, stride=2)
    branch2 = basic_conv(out, num_init_features, kernel_size=1, stride=1)
    branch2 = basic_conv(branch2, num_init_features, kernel_size=3, stride=2)
    branch1 = MaxPooling2D()(out)
    out = concatenate([branch1,branch2],axis=3)
    out = basic_conv(out, num_init_features, kernel_size=1, stride=1)
    
    return out
    
def PeleeNet(growth_rate=32, block_config=[3, 4, 8, 6],num_init_features=32, bottleneck_width=[1, 2, 4, 4], drop_rate=0.05, num_classes=10):


    if type(growth_rate) is list:
            growth_rates = growth_rate
            assert len(growth_rates) == 4, 'The growth rate must be the list and the size must be 4'
    else:
            growth_rates = [growth_rate] * 4

    if type(bottleneck_width) is list:
            bottleneck_widths = bottleneck_width
            assert len(bottleneck_widths) == 4, 'The bottleneck width must be the list and the size must be 4'
    else:
            bottleneck_widths = [bottleneck_width] * 4

    inp = Input((32, 32, 3))
    stem_block =  _StemBlock(inp, num_init_features)
    num_features = num_init_features

    for i, num_layers in enumerate(block_config):
        
            block = _DenseBlock(stem_block,num_layers=num_layers, num_input_features=num_features,
                                bn_size=bottleneck_widths[i], growth_rate=growth_rates[i], drop_rate=drop_rate)
            
            num_features = num_features + num_layers * growth_rates[i]

            block = basic_conv(block, num_features, kernel_size=1, stride=1)
            
            if i != len(block_config) - 1:
                block = AveragePooling2D()(block)
                num_features = num_features
                stem_block = block
            else:
                stem_block =  block
                
    block = Flatten()(block)
    block = Dropout(drop_rate)(block)
#     block = Dense(num_classes, activation='softmax')(block)
    block = Dense(1, activation='sigmoid')(block)

    model = Model(inp,block)
    return model



# if __name__ == '__main__':
#         # test the model
#         model = PeleeNet()
#         model.summary()

In [37]:
model = PeleeNet()

In [38]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_226 (Conv2D)            (None, 16, 16, 32)   896         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_226 (Batch  (None, 16, 16, 32)  128         ['conv2d_226[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_226 (Activation)    (None, 16, 16, 32)   0           ['batch_normalization_226[0

                                                                                                  
 concatenate_49 (Concatenate)   (None, 4, 4, 1152)   0           ['activation_252[0][0]',         
                                                                  'concatenate_48[0][0]',         
                                                                  'activation_255[0][0]']         
                                                                                                  
 activation_257 (Activation)    (None, 4, 4, 1152)   0           ['concatenate_49[0][0]']         
                                                                                                  
 activation_260 (Activation)    (None, 4, 4, 1152)   0           ['concatenate_49[0][0]']         
                                                                                                  
 concatenate_50 (Concatenate)   (None, 4, 4, 3456)   0           ['activation_257[0][0]',         
          

                                )                                                                 
                                                                                                  
 concatenate_58 (Concatenate)   (None, 2, 2, 559872  0           ['activation_298[0][0]',         
                                )                                 'concatenate_57[0][0]',         
                                                                  'activation_301[0][0]']         
                                                                                                  
 activation_303 (Activation)    (None, 2, 2, 559872  0           ['concatenate_58[0][0]']         
                                )                                                                 
                                                                                                  
 activation_306 (Activation)    (None, 2, 2, 559872  0           ['concatenate_58[0][0]']         
          

 flatten_2 (Flatten)            (None, 704)          0           ['activation_338[0][0]']         
                                                                                                  
 dropout_2 (Dropout)            (None, 704)          0           ['flatten_2[0][0]']              
                                                                                                  
 dense_8 (Dense)                (None, 1)            705         ['dropout_2[0][0]']              
                                                                                                  
Total params: 1,125,517,281
Trainable params: 1,125,513,825
Non-trainable params: 3,456
__________________________________________________________________________________________________


In [39]:
model2 = Sequential()
model2.add(model)
#model2.add(GlobalAveragePooling2D())
#model2.add(Flatten())
# model2.add(Dense(1024, activation="relu"))
# model2.add(Dense(512, activation="relu"))

# model2.add(Dense(1024, activation="relu"))
# model2.add(Dense(512, activation="relu"))

# model2.add(Dense(1024, activation="relu"))
# model2.add(Dense(512, activation="relu"))



#model2.add(Dense(1, activation="sigmoid" , name="classification"))

In [40]:
model2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_2 (Functional)        (None, 1)                 1125517281
                                                                 
Total params: 1,125,517,281
Trainable params: 1,125,513,825
Non-trainable params: 3,456
_________________________________________________________________


In [41]:
result_root = r'/Users/krish/ljmu/3.results/29112022-peelenet/'
Path(result_root).mkdir(parents=True, exist_ok=True)

In [42]:
root_path =  r'/Users/krish/ljmu/1.data/afo/'
actual_train = pd.read_csv(root_path+'images/train.csv')

In [43]:
actual_train['humans'] = actual_train.humans.apply(lambda x: str(x))

In [44]:
train_df, valid_df = train_test_split(actual_train, 
                                   test_size = 0.15, 
                                   random_state = 1998,
                                   stratify = actual_train['humans']) # Stratify makes sure that the subset of training and testing data returned keeps the same class propotion of the main dataframe
print('train', train_df.shape[0], 'validation', valid_df.shape[0])

train 2320 validation 410


In [45]:
# Rotation_range - random rotation within the angle mentioned
# Random_shifts - shifts 20% at max. to the horizontal and vertical(reduce to 10% as already diverse data)
# Random_flip - random flips 
# fill_mode - fill the blank pixels, when augmenting with the nearest color
datagen = ImageDataGenerator(rotation_range = 360,
                             width_shift_range=0.1, height_shift_range=0.1,
                             #horizontal_flip=True, vertical_flip=True, # Isnt much useful for us
                             brightness_range=[0.5,1.5],
                             rescale=1./255.,fill_mode='nearest')

train_generator = datagen.flow_from_dataframe(dataframe=train_df, x_col = 'file_name', y_col = 'humans', class_mode='binary', target_size=(224 , 224), batch_size=24)
valid_generator = datagen.flow_from_dataframe(dataframe=valid_df, x_col = 'file_name', y_col = 'humans', class_mode='binary', target_size=(224 , 224), batch_size=24)

Found 2320 validated image filenames belonging to 2 classes.
Found 410 validated image filenames belonging to 2 classes.


In [46]:
model2.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.004,momentum=0.9), 
            loss='BinaryCrossentropy',
            metrics = ['accuracy'])

model2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_2 (Functional)        (None, 1)                 1125517281
                                                                 
Total params: 1,125,517,281
Trainable params: 1,125,513,825
Non-trainable params: 3,456
_________________________________________________________________


In [47]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta = 0.5)

In [48]:
class_weight = {0:train_df['humans'].value_counts()[0]/train_df.shape[0],1:train_df['humans'].value_counts()[1]/train_df.shape[0],}

In [ ]:
history = model2.fit(train_generator, validation_data = valid_generator, epochs = 10, class_weight=class_weight,callbacks=[es])